# Generator Bazy Danych dla Biura Podróży

Ten notebook generuje syntetyczną bazę danych SQLite dla biura podróży. Baza danych zostanie zapisana w folderze `008-03. RAG data`.

## Struktura bazy danych

Baza danych będzie zawierać następujące tabele:
- Klienci (clients) - informacje o klientach biura podróży
- Destynacje (destinations) - miejsca podróży
- Hotele (hotels) - miejsca zakwaterowania w destynacjach
- Wycieczki (tours) - oferty wycieczek do różnych destynacji
- Rezerwacje (bookings) - rezerwacje klientów
- Opinie (reviews) - opinie klientów o wyjazdach
- Pracownicy (employees) - pracownicy biura podróży
- Płatności (payments) - informacje o płatnościach klientów

In [1]:
# Import niezbędnych bibliotek
import os
import pathlib
import sqlite3
import random
import datetime
from faker import Faker
from openai import OpenAI
from pydantic import BaseModel, Field
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
# Konfiguracja
fake = Faker(['pl_PL'])
Faker.seed(42)  # Dla powtarzalnych wyników
random.seed(42)
np.random.seed(42)

# Ścieżka do folderu, w którym zapiszemy bazę danych
output_dir = pathlib.Path('008-03. RAG data')
output_dir.mkdir(exist_ok=True)

# Nazwa pliku bazy danych
db_file = output_dir / 'biuro_podrozy.db'

# Inicjalizacja klienta OpenAI
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

## Definicje modeli Pydantic dla structured outputs

Definiujemy modele Pydantic, które będą służyć do generowania danych za pomocą GPT-4.1-mini.

In [3]:
# Model dla generowania opisów destynacji
class Destination(BaseModel):
    name: str = Field(description="Nazwa destynacji turystycznej (miasto lub region)")
    country: str = Field(description="Kraj, w którym znajduje się destynacja")
    description: str = Field(description="Szczegółowy opis destynacji, zawierający informacje o atrakcjach")
    climate: str = Field(description="Opis klimatu w destynacji")
    best_time_to_visit: str = Field(description="Najlepszy czas na odwiedzenie destynacji")
    language: str = Field(description="Główny język używany w destynacji")
    currency: str = Field(description="Waluta używana w destynacji")

# Model dla listy destynacji
class DestinationList(BaseModel):
    destinations: list[Destination] = Field(description="Lista destynacji turystycznych")

# Model dla generowania opisów hoteli
class Hotel(BaseModel):
    name: str = Field(description="Nazwa hotelu")
    stars: int = Field(description="Liczba gwiazdek hotelu (1-5)")
    description: str = Field(description="Szczegółowy opis hotelu, jego udogodnień i położenia")
    amenities: list[str] = Field(description="Lista udogodnień dostępnych w hotelu")
    room_types: list[str] = Field(description="Typy pokojów dostępnych w hotelu")
    distance_to_center: float = Field(description="Odległość od centrum miasta (w km)")

# Model dla listy hoteli
class HotelList(BaseModel):
    hotels: list[Hotel] = Field(description="Lista hoteli w destynacji")

# Model dla generowania opisów wycieczek
class Tour(BaseModel):
    name: str = Field(description="Nazwa wycieczki")
    description: str = Field(description="Szczegółowy opis wycieczki, jej atrakcji i programu")
    duration_days: int = Field(description="Czas trwania wycieczki w dniach")
    included_services: list[str] = Field(description="Lista usług wliczonych w cenę")
    activities: list[str] = Field(description="Lista aktywności dostępnych podczas wycieczki")
    difficulty_level: str = Field(description="Poziom trudności wycieczki (np. łatwy, średni, trudny)")

# Model dla listy wycieczek
class TourList(BaseModel):
    tours: list[Tour] = Field(description="Lista wycieczek w destynacji")

# Model dla generowania opinii klientów
class Review(BaseModel):
    title: str = Field(description="Tytuł opinii")
    content: str = Field(description="Treść opinii")
    positive_aspects: list[str] = Field(description="Lista pozytywnych aspektów podróży")
    negative_aspects: list[str] = Field(description="Lista negatywnych aspektów podróży (może być pusta)")
    rating: int = Field(description="Ocena w skali 1-10")

# Model dla listy opinii
class ReviewList(BaseModel):
    reviews: list[Review] = Field(description="Lista opinii klientów")

## Tworzenie schematu bazy danych

Tworzymy schemat bazy danych z odpowiednimi tabelami i relacjami.

In [4]:
def create_database_schema():
    """Tworzy schemat bazy danych dla biura podróży"""
    # Usuń bazę danych jeśli istnieje
    if db_file.exists():
        db_file.unlink()
    
    # Połączenie z bazą danych
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Tworzenie tabeli klientów
    cursor.execute('''
    CREATE TABLE clients (
        client_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        phone TEXT,
        address TEXT,
        city TEXT,
        postal_code TEXT,
        date_of_birth DATE,
        registration_date DATE NOT NULL,
        preferred_payment_method TEXT,
        loyalty_points INTEGER DEFAULT 0
    )
    ''')
    
    # Tworzenie tabeli destynacji
    cursor.execute('''
    CREATE TABLE destinations (
        destination_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        country TEXT NOT NULL,
        description TEXT,
        climate TEXT,
        best_time_to_visit TEXT,
        language TEXT,
        currency TEXT,
        popular_score INTEGER,
        average_temperature REAL
    )
    ''')
    
    # Tworzenie tabeli hoteli
    cursor.execute('''
    CREATE TABLE hotels (
        hotel_id INTEGER PRIMARY KEY,
        destination_id INTEGER NOT NULL,
        name TEXT NOT NULL,
        stars INTEGER CHECK (stars BETWEEN 1 AND 5),
        description TEXT,
        amenities TEXT,
        room_types TEXT,
        distance_to_center REAL,
        price_per_night REAL,
        FOREIGN KEY (destination_id) REFERENCES destinations (destination_id)
    )
    ''')
    
    # Tworzenie tabeli wycieczek
    cursor.execute('''
    CREATE TABLE tours (
        tour_id INTEGER PRIMARY KEY,
        destination_id INTEGER NOT NULL,
        name TEXT NOT NULL,
        description TEXT,
        duration_days INTEGER,
        included_services TEXT,
        activities TEXT,
        difficulty_level TEXT,
        base_price REAL,
        availability_start DATE,
        availability_end DATE,
        FOREIGN KEY (destination_id) REFERENCES destinations (destination_id)
    )
    ''')
    
    # Tworzenie tabeli pracowników
    cursor.execute('''
    CREATE TABLE employees (
        employee_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        position TEXT NOT NULL,
        department TEXT,
        email TEXT NOT NULL UNIQUE,
        phone TEXT,
        hire_date DATE,
        salary REAL
    )
    ''')
    
    # Tworzenie tabeli rezerwacji
    cursor.execute('''
    CREATE TABLE bookings (
        booking_id INTEGER PRIMARY KEY,
        client_id INTEGER NOT NULL,
        tour_id INTEGER,
        hotel_id INTEGER,
        employee_id INTEGER,
        booking_date DATE NOT NULL,
        travel_start_date DATE NOT NULL,
        travel_end_date DATE NOT NULL,
        num_adults INTEGER NOT NULL,
        num_children INTEGER DEFAULT 0,
        total_price REAL NOT NULL,
        status TEXT NOT NULL,
        special_requirements TEXT,
        FOREIGN KEY (client_id) REFERENCES clients (client_id),
        FOREIGN KEY (tour_id) REFERENCES tours (tour_id),
        FOREIGN KEY (hotel_id) REFERENCES hotels (hotel_id),
        FOREIGN KEY (employee_id) REFERENCES employees (employee_id)
    )
    ''')
    
    # Tworzenie tabeli płatności
    cursor.execute('''
    CREATE TABLE payments (
        payment_id INTEGER PRIMARY KEY,
        booking_id INTEGER NOT NULL,
        payment_date DATE NOT NULL,
        amount REAL NOT NULL,
        payment_method TEXT NOT NULL,
        status TEXT NOT NULL,
        transaction_id TEXT,
        FOREIGN KEY (booking_id) REFERENCES bookings (booking_id)
    )
    ''')
    
    # Tworzenie tabeli opinii
    cursor.execute('''
    CREATE TABLE reviews (
        review_id INTEGER PRIMARY KEY,
        booking_id INTEGER NOT NULL,
        client_id INTEGER NOT NULL,
        title TEXT,
        content TEXT,
        rating INTEGER CHECK (rating BETWEEN 1 AND 10),
        review_date DATE NOT NULL,
        positive_aspects TEXT,
        negative_aspects TEXT,
        FOREIGN KEY (booking_id) REFERENCES bookings (booking_id),
        FOREIGN KEY (client_id) REFERENCES clients (client_id)
    )
    ''')
    
    conn.commit()
    conn.close()
    
    print(f"Schema bazy danych został utworzony w pliku: {db_file}")

create_database_schema()

Schema bazy danych został utworzony w pliku: 008-03. RAG data/biuro_podrozy.db


## Funkcje pomocnicze

Tworzymy funkcje pomocnicze do generowania danych.

In [23]:
def generate_with_gpt(prompt, model_format, batch_size=1, retries=3, delay=1):
    """
    Generuje dane przy użyciu GPT-4.1-mini w określonym formacie Pydantic.
    Może generować pojedyncze rekordy lub całe batche danych.
    
    Args:
        prompt (str): Prompt dla modelu GPT
        model_format (BaseModel): Klasa Pydantic definiująca format odpowiedzi
        batch_size (int): Liczba rekordów do wygenerowania w jednym wywołaniu
        retries (int): Liczba prób w przypadku błędu
        delay (int): Opóźnienie między próbami (w sekundach)
        
    Returns:
        List of instances of model_format: Lista wygenerowanych rekordów
    """
    # Ustawienie odpowiedniego modelu listowego na podstawie typu modelu
    if model_format == Destination:
        response_format = DestinationList
    elif model_format == Hotel:
        response_format = HotelList
    elif model_format == Tour:
        response_format = TourList
    elif model_format == Review:
        response_format = ReviewList
    else:
        raise ValueError(f"Nieobsługiwany format modelu: {model_format}")
    
    batch_prompt = f"""
    {prompt}
    
    Wygeneruj {batch_size} różnych rekordów. Każdy powinien być unikalny i zawierać wszystkie wymagane pola.
    """
        
    response = client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "Jesteś ekspertem od turystyki. Generujesz szczegółowe, realistyczne dane w języku polskim."},
            {"role": "user", "content": batch_prompt},
        ],
        response_format=response_format,
        temperature=0.7,
        max_tokens=32000
    )
    
    # Wyciągnij odpowiednią listę z modelu listowego
    if model_format == Destination:
        return response.choices[0].message.parsed.destinations
    elif model_format == Hotel:
        return response.choices[0].message.parsed.hotels
    elif model_format == Tour:
        return response.choices[0].message.parsed.tours
    elif model_format == Review:
        return response.choices[0].message.parsed.reviews

def random_range(min_val, max_val):
    """
    Zwraca losową liczbę z zakresu przy zachowaniu proporcji min 1:3.
    Jeśli podany zakres ma mniejszą proporcję, dostosowuje go.
    """
    if min_val > 0 and max_val / min_val < 3:
        max_val = min_val * 3
    return random.randint(min_val, max_val)

def generate_date_between(start_date, end_date):
    """
    Generuje losową datę między start_date a end_date.
    """
    days_diff = (end_date - start_date).days
    random_days = random.randint(0, days_diff)
    return start_date + datetime.timedelta(days=random_days)

def list_to_str(lst):
    """
    Konwertuje listę do stringa rozdzielonego przecinkami.
    """
    return ','.join(str(item) for item in lst) if lst else ''

## Generowanie danych

Zaczynamy od generowania podstawowych danych dla naszej bazy.

### 1. Generowanie destynacji

Najpierw wygenerujemy dane o destynacjach turystycznych przy użyciu GPT-4.1-mini.

In [6]:
def generate_destinations(num_destinations=30):
    """
    Generuje dane o destynacjach turystycznych przy użyciu GPT-4.1-mini w grupach.
    
    Args:
        num_destinations (int): Liczba destynacji do wygenerowania
        
    Returns:
        list: Lista wygenerowanych destynacji
    """
    destinations = []
    
    continents = [
        "Europa", "Azja", "Ameryka Północna", "Ameryka Południowa", 
        "Afryka", "Australia i Oceania"
    ]
    
    # Definiujemy wielkość grupy (batch)
    batch_size = 5  # Generujemy 5 destynacji w jednym wywołaniu
    
    # Generujemy destynacje w batchu
    for i in tqdm(range(0, num_destinations, batch_size), desc="Generowanie destynacji"):
        current_batch_size = min(batch_size, num_destinations - i)
        
        # Wybierz kontynenty dla tej partii destynacji
        continents_for_batch = [continents[(i + j) % len(continents)] for j in range(current_batch_size)]
        continent_str = ", ".join(continents_for_batch)
        
        # Lista już wygenerowanych destynacji do przekazania w prompcie
        existing_destinations = ", ".join([d['name'] + " (" + d['country'] + ")" for d in destinations])
        
        prompt = f"""
        Wygeneruj szczegółowe informacje o atrakcyjnych destynacjach turystycznych na kontynentach: {continent_str}.
        Wymyśl miejsca, które będą interesujące dla turystów i zawierające wszystkie wymagane informacje.
        Użyj języka polskiego. Dodaj szczegółowe opisy miejsc i ich atrakcji.
        
        Już wygenerowano następujące destynacje (unikaj ich): {existing_destinations}
        """
        
        batch_destinations = generate_with_gpt(prompt, Destination, batch_size=current_batch_size)
        
        # Dodajemy dodatkowe pola, których nie generuje GPT
        for destination in batch_destinations:
            destination_dict = destination.model_dump()
            destination_dict['popular_score'] = random.randint(1, 100)
            destination_dict['average_temperature'] = round(random.uniform(0, 35), 1)
            
            destinations.append(destination_dict)
    
    return destinations

def insert_destinations(destinations):
    """
    Wstawia wygenerowane destynacje do bazy danych.
    
    Args:
        destinations (list): Lista słowników z danymi destynacji
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    for destination in tqdm(destinations, desc="Zapisywanie destynacji do bazy"):
        cursor.execute("""
        INSERT INTO destinations 
        (name, country, description, climate, best_time_to_visit, language, currency, popular_score, average_temperature)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            destination['name'],
            destination['country'],
            destination['description'],
            destination['climate'],
            destination['best_time_to_visit'],
            destination['language'],
            destination['currency'],
            destination['popular_score'],
            destination['average_temperature']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(destinations)} destynacji do bazy danych.")

In [8]:
# Generowanie destynacji - zakomentowane, aby nie generować danych przy każdym uruchomieniu
num_destinations = 30  # Liczba destynacji do wygenerowania
destinations = generate_destinations(num_destinations)
insert_destinations(destinations)

Zapisywanie destynacji do bazy: 100%|██████████| 30/30 [00:00<00:00, 20560.31it/s]

Zapisano 30 destynacji do bazy danych.


### 2. Generowanie hoteli

Dla każdej destynacji wygenerujemy różną liczbę hoteli.

In [9]:
def generate_hotels_for_destination(destination_id, destination_name, min_hotels=2, max_hotels=8):
    """
    Generuje dane o hotelach dla konkretnej destynacji przy użyciu GPT-4.1-mini.
    Używa trybu wsadowego do generowania wielu hoteli na raz.
    
    Args:
        destination_id (int): ID destynacji
        destination_name (str): Nazwa destynacji dla prompta
        min_hotels (int): Minimalna liczba hoteli do wygenerowania
        max_hotels (int): Maksymalna liczba hoteli do wygenerowania
        
    Returns:
        list: Lista wygenerowanych hoteli
    """
    # Używamy random_range zamiast stałej liczby hoteli - zgodnie z wymogami
    num_hotels = random_range(min_hotels, max_hotels)
    
    # Użyj wielkości batcha jako 3, aby nie przeciążać modelu dla jednej destynacji
    batch_size = max_hotels
    hotels = []
    
    for i in range(0, num_hotels, batch_size):
        current_batch_size = min(batch_size, num_hotels - i)
        
        prompt = f"""
        Wygeneruj szczegółowe informacje o hotelach w destynacji {destination_name}.
        Wymyśl realistyczne hotele z unikalnymi nazwami, opisami i udogodnieniami.
        Dla każdego hotelu przydziel gwiazdki od 1 do 5, gdzie 5 to hotel luksusowy.
        Każdy hotel powinien mieć inną nazwę i być innego typu (np. biznesowy, rodzinny, budżetowy).
        Użyj języka polskiego w opisie.
        """
        
        batch_hotels = generate_with_gpt(prompt, Hotel, batch_size=current_batch_size)
        
        # Dodajemy dodatkowe pola i dołączamy do głównej listy
        for hotel in batch_hotels:
            hotel_dict = hotel.model_dump()
            hotel_dict['destination_id'] = destination_id
            hotel_dict['amenities'] = list_to_str(hotel_dict['amenities'])
            hotel_dict['room_types'] = list_to_str(hotel_dict['room_types'])
            hotel_dict['price_per_night'] = round(hotel.stars * random.uniform(100, 300), 2)
            
            hotels.append(hotel_dict)
    
    return hotels

In [10]:
def generate_and_insert_hotels():
    """
    Generuje i wstawia hotele do bazy danych dla wszystkich destynacji.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Pobierz wszystkie destynacje
    cursor.execute("SELECT destination_id, name FROM destinations")
    destinations = cursor.fetchall()
    
    all_hotels = []
    
    for destination_id, destination_name in tqdm(destinations, desc="Generowanie hoteli"):
        hotels = generate_hotels_for_destination(destination_id, destination_name)
        all_hotels.extend(hotels)
        
    # Wstaw hotele do bazy
    for hotel in tqdm(all_hotels, desc="Zapisywanie hoteli do bazy"):
        cursor.execute("""
        INSERT INTO hotels 
        (destination_id, name, stars, description, amenities, room_types, distance_to_center, price_per_night)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            hotel['destination_id'],
            hotel['name'],
            hotel['stars'],
            hotel['description'],
            hotel['amenities'],
            hotel['room_types'],
            hotel['distance_to_center'],
            hotel['price_per_night']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(all_hotels)} hoteli do bazy danych.")

In [11]:
# Generowanie hoteli
generate_and_insert_hotels()

Zapisywanie hoteli do bazy: 100%|██████████| 145/145 [00:00<00:00, 28448.60it/s]

Zapisano 145 hoteli do bazy danych.


### 3. Generowanie wycieczek

Dla każdej destynacji wygenerujemy różną liczbę wycieczek.

In [12]:
def generate_tours_for_destination(destination_id, destination_name, min_tours=1, max_tours=5):
    """
    Generuje dane o wycieczkach dla konkretnej destynacji przy użyciu GPT-4.1-mini.
    Używa trybu wsadowego do generowania wielu wycieczek na raz.
    
    Args:
        destination_id (int): ID destynacji
        destination_name (str): Nazwa destynacji dla prompta
        min_tours (int): Minimalna liczba wycieczek
        max_tours (int): Maksymalna liczba wycieczek
        
    Returns:
        list: Lista wygenerowanych wycieczek
    """
    # Używamy random_range zamiast stałej liczby wycieczek
    num_tours = random_range(min_tours, max_tours)
    
    # Użyj batcha zamiast generowania pojedynczych wycieczek
    batch_size = num_tours  # Generujemy wszystkie wycieczki na raz, bo zwykle jest ich mało (1-5)
    
    prompt = f"""
    Wygeneruj szczegółowe informacje o wycieczkach w destynacji {destination_name}.
    Wymyśl unikalne nazwy i szczegółowe opisy wycieczek, programy zwiedzania, zawarte usługi.
    Każda wycieczka powinna mieć inny charakter (np. rodzinna, przygodowa, kulturowa, relaksacyjna).
    Użyj języka polskiego w opisie.
    """
    
    batch_tours = generate_with_gpt(prompt, Tour, batch_size=batch_size)
    
    tours = []
    for tour in batch_tours:
        tour_dict = tour.model_dump()
        tour_dict['destination_id'] = destination_id
        tour_dict['included_services'] = list_to_str(tour_dict['included_services'])
        tour_dict['activities'] = list_to_str(tour_dict['activities'])
        
        # Losowe ceny bazowe zależne od długości wycieczki
        tour_dict['base_price'] = round(tour.duration_days * random.uniform(300, 800), 2)
        
        # Ustawienie dat dostępności
        today = datetime.date.today()
        # Losowe daty dostępności na przestrzeni roku
        start_date = today + datetime.timedelta(days=random.randint(7, 90))
        end_date = start_date + datetime.timedelta(days=random.randint(90, 365))
        tour_dict['availability_start'] = start_date.isoformat()
        tour_dict['availability_end'] = end_date.isoformat()
        
        tours.append(tour_dict)
    
    return tours

In [13]:
def generate_and_insert_tours():
    """
    Generuje i wstawia wycieczki do bazy danych dla wszystkich destynacji.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Pobierz wszystkie destynacje
    cursor.execute("SELECT destination_id, name FROM destinations")
    destinations = cursor.fetchall()
    
    all_tours = []
    
    for destination_id, destination_name in tqdm(destinations, desc="Generowanie wycieczek"):
        tours = generate_tours_for_destination(destination_id, destination_name)
        all_tours.extend(tours)
        
    # Wstaw wycieczki do bazy
    for tour in tqdm(all_tours, desc="Zapisywanie wycieczek do bazy"):
        cursor.execute("""
        INSERT INTO tours 
        (destination_id, name, description, duration_days, included_services, activities, 
         difficulty_level, base_price, availability_start, availability_end)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            tour['destination_id'],
            tour['name'],
            tour['description'],
            tour['duration_days'],
            tour['included_services'],
            tour['activities'],
            tour['difficulty_level'],
            tour['base_price'],
            tour['availability_start'],
            tour['availability_end']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(all_tours)} wycieczek do bazy danych.")

In [14]:
# Generowanie wycieczek
generate_and_insert_tours()

Zapisywanie wycieczek do bazy: 100%|██████████| 89/89 [00:00<00:00, 17833.61it/s]

Zapisano 89 wycieczek do bazy danych.


### 4. Generowanie pracowników

Generujemy dane pracowników biura podróży.

In [15]:
def generate_employees(min_employees=20, max_employees=25):
    """
    Generuje dane pracowników biura podróży.
    
    Args:
        min_employees (int): Minimalna liczba pracowników
        max_employees (int): Maksymalna liczba pracowników
        
    Returns:
        list: Lista wygenerowanych pracowników
    """
    # Używamy random_range dla liczby pracowników
    num_employees = random_range(min_employees, max_employees)
    
    positions = [
        "Doradca turystyczny", "Specjalista ds. rezerwacji", "Kierownik biura", "Asystent sprzedaży", 
        "Agent turystyczny", "Specjalista ds. obsługi klienta", "Koordynator wyjazdów", "Księgowy",
        "Specjalista ds. marketingu", "Pilot wycieczek", "Przewodnik turystyczny", "Dyrektor sprzedaży"
    ]
    
    departments = [
        "Sprzedaż", "Obsługa klienta", "Administracja", "Marketing", "Księgowość", "Zarząd", "Logistyka"
    ]
    
    employees = []
    used_emails = set()  # Zapobiega duplikatom emaili
    
    for i in range(num_employees):
        first_name = fake.first_name()
        last_name = fake.last_name()
        
        position = random.choice(positions)
        department = random.choice(departments)
        
        # Tworzenie unikalnego adresu email
        email_base = f"{first_name.lower()}.{last_name.lower()}@biuropodrozy.pl".replace(' ', '')
        email = email_base
        counter = 1
        
        # Jeśli email jest już użyty, dodaj licznik
        while email in used_emails:
            email = f"{email_base.split('@')[0]}{counter}@{email_base.split('@')[1]}"
            counter += 1
            
        used_emails.add(email)
        
        # Losowa data zatrudnienia z ostatnich 10 lat
        hire_date = fake.date_between(start_date='-10y', end_date='today')
        
        # Wynagrodzenie zależne od stanowiska
        if "Kierownik" in position or "Dyrektor" in position:
            salary = round(random.uniform(8000, 15000), 2)
        elif "Specjalista" in position or "Koordynator" in position:
            salary = round(random.uniform(5000, 8000), 2)
        else:
            salary = round(random.uniform(4000, 6000), 2)
        
        employees.append({
            'first_name': first_name,
            'last_name': last_name,
            'position': position,
            'department': department,
            'email': email,
            'phone': fake.phone_number(),
            'hire_date': hire_date,
            'salary': salary
        })
    
    return employees

def insert_employees(employees):
    """
    Wstawia wygenerowanych pracowników do bazy danych.
    
    Args:
        employees (list): Lista słowników z danymi pracowników
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    for employee in tqdm(employees, desc="Zapisywanie pracowników do bazy"):
        cursor.execute("""
        INSERT INTO employees 
        (first_name, last_name, position, department, email, phone, hire_date, salary)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            employee['first_name'],
            employee['last_name'],
            employee['position'],
            employee['department'],
            employee['email'],
            employee['phone'],
            employee['hire_date'],
            employee['salary']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(employees)} pracowników do bazy danych.")

In [16]:
# Generowanie pracowników
employees = generate_employees()
insert_employees(employees)

Zapisywanie pracowników do bazy: 100%|██████████| 50/50 [00:00<00:00, 18141.45it/s]

Zapisano 50 pracowników do bazy danych.


### 5. Generowanie klientów

Generujemy dane klientów biura podróży.

In [19]:
def generate_clients(min_clients=100, max_clients=300):
    """
    Generuje dane klientów biura podróży.
    
    Args:
        min_clients (int): Minimalna liczba klientów
        max_clients (int): Maksymalna liczba klientów
        
    Returns:
        list: Lista wygenerowanych klientów
    """
    # Używamy random_range dla liczby klientów
    num_clients = random_range(min_clients, max_clients)
    
    payment_methods = ["Karta kredytowa", "Gotówka", "Przelew bankowy", "BLIK", "PayPal"]
    
    clients = []
    used_emails = set()  # Zapobiega duplikatom emaili
    
    for i in range(num_clients):
        first_name = fake.first_name()
        last_name = fake.last_name()
        
        # Tworzenie unikalnego adresu email
        email_providers = ["gmail.com", "wp.pl", "onet.pl", "interia.pl", "o2.pl", "yahoo.com"]
        email_base = f"{first_name.lower()}.{last_name.lower()}@{random.choice(email_providers)}".replace(' ', '')
        email = email_base
        counter = 1
        
        while email in used_emails:
            email = f"{email_base.split('@')[0]}{counter}@{email_base.split('@')[1]}"
            counter += 1
            
        used_emails.add(email)
        
        # Data urodzenia - dorośli od 18 do 80 lat
        date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=80)
        
        # Data rejestracji - w ciągu ostatnich 5 lat
        registration_date = fake.date_between(start_date='-5y', end_date='today')
        
        # Punkty lojalnościowe
        loyalty_points = random.randint(0, 5000)
        
        clients.append({
            'first_name': first_name,
            'last_name': last_name,
            'email': email,
            'phone': fake.phone_number(),
            'address': fake.street_address(),
            'city': fake.city(),
            'postal_code': fake.postcode(),
            'date_of_birth': date_of_birth,
            'registration_date': registration_date,
            'preferred_payment_method': random.choice(payment_methods),
            'loyalty_points': loyalty_points
        })
    
    return clients

def insert_clients(clients):
    """
    Wstawia wygenerowanych klientów do bazy danych.
    
    Args:
        clients (list): Lista słowników z danymi klientów
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    for client in tqdm(clients, desc="Zapisywanie klientów do bazy"):
        cursor.execute("""
        INSERT INTO clients 
        (first_name, last_name, email, phone, address, city, postal_code, 
         date_of_birth, registration_date, preferred_payment_method, loyalty_points)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            client['first_name'],
            client['last_name'],
            client['email'],
            client['phone'],
            client['address'],
            client['city'],
            client['postal_code'],
            client['date_of_birth'],
            client['registration_date'],
            client['preferred_payment_method'],
            client['loyalty_points']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(clients)} klientów do bazy danych.")

In [20]:
# Generowanie klientów
clients = generate_clients()
insert_clients(clients)

Zapisywanie klientów do bazy: 100%|██████████| 181/181 [00:00<00:00, 51884.16it/s]

Zapisano 181 klientów do bazy danych.


### 6. Generowanie rezerwacji

Generujemy dane rezerwacji wycieczek dla różnych klientów.

In [21]:
def generate_bookings(min_bookings_per_client=0, max_bookings_per_client=4):
    """
    Generuje dane rezerwacji dla klientów.
    
    Args:
        min_bookings_per_client (int): Minimalna liczba rezerwacji per klient
        max_bookings_per_client (int): Maksymalna liczba rezerwacji per klient
        
    Returns:
        list: Lista wygenerowanych rezerwacji
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Pobieranie danych z bazy
    cursor.execute("SELECT client_id FROM clients")
    client_ids = [row[0] for row in cursor.fetchall()]
    
    cursor.execute("SELECT tour_id, destination_id, duration_days, base_price FROM tours")
    tours = cursor.fetchall()
    tour_dict = {tour[0]: (tour[1], tour[2], tour[3]) for tour in tours}  # tour_id: (destination_id, duration_days, base_price)
    
    cursor.execute("SELECT hotel_id, destination_id, price_per_night FROM hotels")
    hotels = cursor.fetchall()
    hotel_dict = {}
    hotels_by_destination = {}
    
    for hotel_id, destination_id, price_per_night in hotels:
        hotel_dict[hotel_id] = (destination_id, price_per_night)
        if destination_id not in hotels_by_destination:
            hotels_by_destination[destination_id] = []
        hotels_by_destination[destination_id].append(hotel_id)
    
    cursor.execute("SELECT employee_id FROM employees WHERE position LIKE '%rezerwacji%' OR position LIKE '%Doradca%' OR position LIKE '%Agent%'")
    booking_employees = [row[0] for row in cursor.fetchall()]
    if not booking_employees:  # Jeśli nie znaleziono odpowiednich pracowników
        cursor.execute("SELECT employee_id FROM employees LIMIT 5")
        booking_employees = [row[0] for row in cursor.fetchall()]
    
    bookings = []
    
    # Dla każdego klienta generujemy różną liczbę rezerwacji
    for client_id in tqdm(client_ids, desc="Generowanie rezerwacji"):
        # Używamy random_range dla liczby rezerwacji per klient
        num_bookings = random_range(min_bookings_per_client, max_bookings_per_client)
        
        if num_bookings == 0:
            continue  # Niektórzy klienci mogą nie mieć rezerwacji
        
        for i in range(num_bookings):
            # Losowe wybieranie wycieczki
            tour_id = random.choice(list(tour_dict.keys()))
            destination_id, duration_days, tour_base_price = tour_dict[tour_id]
            
            # Losowy hotel w tej samej destynacji
            if destination_id in hotels_by_destination and hotels_by_destination[destination_id]:
                hotel_id = random.choice(hotels_by_destination[destination_id])
                _, hotel_price_per_night = hotel_dict[hotel_id]
            else:
                hotel_id = None
                hotel_price_per_night = 0
            
            # Losowy pracownik obsługujący
            employee_id = random.choice(booking_employees)
            
            # Data rezerwacji - ostatnie 2 lata
            booking_date = fake.date_between(start_date='-2y', end_date='today')
            
            # Data rozpoczęcia podróży - od 1 miesiąca do 1 roku po rezerwacji
            travel_start_date = booking_date + datetime.timedelta(days=random.randint(30, 365))
            
            # Data zakończenia podróży
            travel_end_date = travel_start_date + datetime.timedelta(days=duration_days)
            
            # Liczba osób
            num_adults = random.randint(1, 4)
            num_children = random.randint(0, 3)
            
            # Obliczanie ceny
            tour_total = tour_base_price * (num_adults + num_children * 0.6)  # Dzieci mają zniżkę 40%
            hotel_total = hotel_price_per_night * duration_days * (num_adults + num_children * 0.3) if hotel_id else 0
            total_price = round(tour_total + hotel_total, 2)
            
            # Status rezerwacji
            if travel_end_date < datetime.date.today():
                status = "Zakończona"
            elif travel_start_date < datetime.date.today():
                status = "W trakcie"
            elif travel_start_date - datetime.timedelta(days=7) < datetime.date.today():
                status = "Potwierdzona"
            else:
                status = random.choice(["Potwierdzona", "Oczekująca na płatność", "Wstępna"])
            
            # Specjalne wymagania
            special_requirements = None
            if random.random() < 0.3:  # 30% rezerwacji ma specjalne wymagania
                requirements = [
                    "Pokój dla niepalących",
                    "Dieta wegetariańska",
                    "Pokój z widokiem na morze",
                    "Dodatkowe łóżko dla dziecka",
                    "Późne zameldowanie",
                    "Usługa transferu z lotniska",
                    "Pokój przystosowany dla osób niepełnosprawnych",
                    "Preferencje co do piętra (wysoko/nisko)",
                    "Dieta bezglutenowa",
                    "Łóżeczko dla niemowlaka"
                ]
                special_requirements = ", ".join(random.sample(requirements, k=random.randint(1, 3)))
            
            bookings.append({
                'client_id': client_id,
                'tour_id': tour_id,
                'hotel_id': hotel_id,
                'employee_id': employee_id,
                'booking_date': booking_date,
                'travel_start_date': travel_start_date,
                'travel_end_date': travel_end_date,
                'num_adults': num_adults,
                'num_children': num_children,
                'total_price': total_price,
                'status': status,
                'special_requirements': special_requirements
            })
    
    conn.close()
    return bookings

def insert_bookings(bookings):
    """
    Wstawia wygenerowane rezerwacje do bazy danych.
    
    Args:
        bookings (list): Lista słowników z danymi rezerwacji
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    for booking in tqdm(bookings, desc="Zapisywanie rezerwacji do bazy"):
        cursor.execute("""
        INSERT INTO bookings 
        (client_id, tour_id, hotel_id, employee_id, booking_date, travel_start_date, 
         travel_end_date, num_adults, num_children, total_price, status, special_requirements)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            booking['client_id'],
            booking['tour_id'],
            booking['hotel_id'],
            booking['employee_id'],
            booking['booking_date'],
            booking['travel_start_date'],
            booking['travel_end_date'],
            booking['num_adults'],
            booking['num_children'],
            booking['total_price'],
            booking['status'],
            booking['special_requirements']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(bookings)} rezerwacji do bazy danych.")

In [24]:
# Generowanie rezerwacji
bookings = generate_bookings()
insert_bookings(bookings)

Zapisywanie rezerwacji do bazy: 100%|██████████| 572/572 [00:00<00:00, 208240.77it/s]

Zapisano 572 rezerwacji do bazy danych.


### 7. Generowanie płatności

Dla każdej rezerwacji generujemy odpowiednie płatności.

In [25]:
def generate_payments():
    """
    Generuje dane o płatnościach dla istniejących rezerwacji.
    
    Returns:
        list: Lista wygenerowanych płatności
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Pobierz wszystkie rezerwacje
    cursor.execute("""
    SELECT booking_id, client_id, total_price, booking_date, status 
    FROM bookings
    """)
    booking_data = cursor.fetchall()
    
    # Pobierz preferowane metody płatności klientów
    cursor.execute("SELECT client_id, preferred_payment_method FROM clients")
    client_payment_methods = {row[0]: row[1] for row in cursor.fetchall()}
    
    payments = []
    
    for booking_id, client_id, total_price, booking_date, status in tqdm(booking_data, desc="Generowanie płatności"):
        # Konwertuj daty z string na obiekt datetime.date
        if isinstance(booking_date, str):
            booking_date = datetime.date.fromisoformat(booking_date)
        
        # Preferowana metoda płatności klienta
        payment_method = client_payment_methods.get(client_id, "Karta kredytowa")
        
        # Statusy płatności zależne od statusu rezerwacji
        if status in ["Zakończona", "W trakcie", "Potwierdzona"]:
            # Pełna płatność lub raty
            if total_price > 5000 and random.random() < 0.7:  # 70% droższych wycieczek z płatnością w ratach
                # Płatność w ratach
                num_installments = random.randint(2, 4)
                installment_amount = total_price / num_installments
                
                # Pierwsza rata przy rezerwacji
                first_payment_date = booking_date
                payments.append({
                    'booking_id': booking_id,
                    'payment_date': first_payment_date,
                    'amount': round(installment_amount, 2),
                    'payment_method': payment_method,
                    'status': "Zaksięgowana",
                    'transaction_id': fake.uuid4()[:13]
                })
                
                # Pozostałe raty
                for i in range(1, num_installments):
                    next_payment_date = booking_date + datetime.timedelta(days=30*i)
                    
                    # Jeśli data płatności jest w przyszłości, płatność może być oczekująca
                    if next_payment_date > datetime.date.today():
                        payment_status = "Oczekująca"
                        transaction_id = None
                    else:
                        payment_status = "Zaksięgowana"
                        transaction_id = fake.uuid4()[:13]
                    
                    payments.append({
                        'booking_id': booking_id,
                        'payment_date': next_payment_date,
                        'amount': round(installment_amount, 2),
                        'payment_method': payment_method,
                        'status': payment_status,
                        'transaction_id': transaction_id
                    })
            else:
                # Jednorazowa płatność
                payments.append({
                    'booking_id': booking_id,
                    'payment_date': booking_date + datetime.timedelta(days=random.randint(0, 5)),
                    'amount': total_price,
                    'payment_method': payment_method,
                    'status': "Zaksięgowana",
                    'transaction_id': fake.uuid4()[:13]
                })
        elif status == "Oczekująca na płatność":
            # Zaliczka
            deposit_amount = round(total_price * random.uniform(0.1, 0.3), 2)
            payments.append({
                'booking_id': booking_id,
                'payment_date': booking_date,
                'amount': deposit_amount,
                'payment_method': payment_method,
                'status': "Zaksięgowana",
                'transaction_id': fake.uuid4()[:13]
            })
            
            # Pozostała kwota oczekująca
            remaining_amount = round(total_price - deposit_amount, 2)
            payments.append({
                'booking_id': booking_id,
                'payment_date': datetime.date.today() + datetime.timedelta(days=random.randint(7, 30)),
                'amount': remaining_amount,
                'payment_method': payment_method,
                'status': "Oczekująca",
                'transaction_id': None
            })
        elif status == "Wstępna":
            # Brak płatności dla rezerwacji wstępnych
            pass
    
    conn.close()
    return payments

def insert_payments(payments):
    """
    Wstawia wygenerowane płatności do bazy danych.
    
    Args:
        payments (list): Lista słowników z danymi płatności
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    for payment in tqdm(payments, desc="Zapisywanie płatności do bazy"):
        cursor.execute("""
        INSERT INTO payments 
        (booking_id, payment_date, amount, payment_method, status, transaction_id)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (
            payment['booking_id'],
            payment['payment_date'],
            payment['amount'],
            payment['payment_method'],
            payment['status'],
            payment['transaction_id']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(payments)} płatności do bazy danych.")

In [26]:
# Generowanie płatności
payments = generate_payments()
insert_payments(payments)

Zapisywanie płatności do bazy: 100%|██████████| 1126/1126 [00:00<00:00, 238633.03it/s]

Zapisano 1126 płatności do bazy danych.


### 8. Generowanie opinii

Generujemy opinie klientów dla zakończonych rezerwacji przy użyciu GPT-4.1-mini.

In [29]:
def generate_reviews():
    """
    Generuje opinie dla zakończonych rezerwacji przy użyciu GPT-4.1-mini.
    Używa trybu wsadowego do generowania wielu opinii na raz.
    
    Returns:
        list: Lista wygenerowanych opinii
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Pobierz zakończone rezerwacje
    cursor.execute("""
    SELECT b.booking_id, b.client_id, b.travel_end_date, t.name AS tour_name, t.description, 
           d.name AS destination_name, d.country, c.first_name, c.last_name, h.name AS hotel_name, h.stars
    FROM bookings b
    JOIN tours t ON b.tour_id = t.tour_id
    JOIN destinations d ON t.destination_id = d.destination_id
    JOIN clients c ON b.client_id = c.client_id
    LEFT JOIN hotels h ON b.hotel_id = h.hotel_id
    WHERE b.status = 'Zakończona'
    """)
    
    completed_bookings = cursor.fetchall()
    
    # Jeśli nie ma zakończonych rezerwacji, symulujemy je na potrzeby opinii
    if not completed_bookings:
        cursor.execute("""
        SELECT b.booking_id, b.client_id, b.travel_start_date, t.name AS tour_name, t.description, 
               d.name AS destination_name, d.country, c.first_name, c.last_name, h.name AS hotel_name, h.stars
        FROM bookings b
        JOIN tours t ON b.tour_id = t.tour_id
        JOIN destinations d ON t.destination_id = d.destination_id
        JOIN clients c ON b.client_id = c.client_id
        LEFT JOIN hotels h ON b.hotel_id = h.hotel_id
        LIMIT 20
        """)
        completed_bookings = cursor.fetchall()
    
    # Filtrujemy rezerwacje (nie wszyscy klienci piszą opinie)
    review_bookings = []
    for booking in completed_bookings:
        if random.random() > 0.3:  # 70% klientów pisze opinie
            review_bookings.append(booking)
    
    reviews = []
    
    # Przetwarzanie rezerwacji w grupach (batch) po 5
    batch_size = 5
    
    for i in tqdm(range(0, len(review_bookings), batch_size), desc="Generowanie opinii"):
        current_batch = review_bookings[i:i+batch_size]
        current_batch_size = len(current_batch)
        
        batch_prompts = []
        batch_metadata = []
        
        for booking in current_batch:
            booking_id, client_id, travel_end_date, tour_name, tour_description = booking[:5]
            destination_name, country, first_name, last_name = booking[5:9]
            hotel_name, hotel_stars = booking[9:11] if len(booking) > 10 else ("Nieznany", 0)
            
            # Format daty
            if isinstance(travel_end_date, str):
                travel_end_date = datetime.date.fromisoformat(travel_end_date)
            
            # Data dodania opinii - od 1 do 30 dni po zakończeniu podróży
            review_date = travel_end_date + datetime.timedelta(days=random.randint(1, 30))
            
            prompt = f"""
            Wygeneruj opinię dla klienta {first_name} {last_name}, który odbył wycieczkę "{tour_name}" 
            do {destination_name}, {country}. {f'Nocował w hotelu {hotel_name} ({hotel_stars} gwiazdek).' if hotel_name != 'Nieznany' else ''}
            
            Opis wycieczki: {tour_description[:200]}...
            
            Wygeneruj realistyczną opinię z tytułem, oceną (1-10), pozytywnymi i negatywnymi aspektami podróży.
            Opinia powinna być w języku polskim. Uwzględnij zarówno pochwały, jak i konstruktywną krytykę.
            """
            
            batch_prompts.append(prompt)
            batch_metadata.append({
                'booking_id': booking_id,
                'client_id': client_id,
                'review_date': review_date
            })
        
        # Jeden prompt zawierający wszystkie wytyczne dla batcha
        combined_prompt = f"""
        Wygeneruj {current_batch_size} różnych opinii klientów dla wycieczek:
        
        {' '.join([f"Opinia {j+1}: " + prompt for j, prompt in enumerate(batch_prompts)])}
        
        Każda opinia powinna zawierać tytuł, treść, ocenę oraz listę pozytywnych i negatywnych aspektów.
        """
        
        try:
            batch_reviews = generate_with_gpt(combined_prompt, Review, batch_size=current_batch_size)
            
            # Połącz wyniki z metadanymi
            for review_data, metadata in zip(batch_reviews, batch_metadata):
                reviews.append({
                    'booking_id': metadata['booking_id'],
                    'client_id': metadata['client_id'],
                    'title': review_data.title,
                    'content': review_data.content,
                    'rating': review_data.rating,
                    'review_date': metadata['review_date'],
                    'positive_aspects': list_to_str(review_data.positive_aspects),
                    'negative_aspects': list_to_str(review_data.negative_aspects)
                })
        except Exception as e:
            print(f"Błąd podczas generowania opinii w batchu: {e}")
            continue
    
    conn.close()
    return reviews

def insert_reviews(reviews):
    """
    Wstawia wygenerowane opinie do bazy danych.
    
    Args:
        reviews (list): Lista słowników z danymi opinii
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    for review in tqdm(reviews, desc="Zapisywanie opinii do bazy"):
        cursor.execute("""
        INSERT INTO reviews 
        (booking_id, client_id, title, content, rating, review_date, positive_aspects, negative_aspects)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            review['booking_id'],
            review['client_id'],
            review['title'],
            review['content'],
            review['rating'],
            review['review_date'],
            review['positive_aspects'],
            review['negative_aspects']
        ))
    
    conn.commit()
    conn.close()
    print(f"Zapisano {len(reviews)} opinii do bazy danych.")

In [30]:
# Generowanie opinii
reviews = generate_reviews()
insert_reviews(reviews)

Zapisywanie opinii do bazy: 100%|██████████| 299/299 [00:00<00:00, 67218.57it/s]

Zapisano 299 opinii do bazy danych.


## Podsumowanie i weryfikacja danych

Sprawdźmy statystyki wygenerowanej bazy danych.

In [31]:
def show_database_stats():
    """
    Wyświetla statystyki wygenerowanej bazy danych.
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    tables = [
        "clients", "destinations", "hotels", "tours", 
        "employees", "bookings", "payments", "reviews"
    ]
    
    print("Statystyki bazy danych biura podróży:")
    print("-" * 40)
    
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"{table.capitalize()}: {count} rekordów")
    
    # Kilka dodatkowych statystyk
    cursor.execute("SELECT AVG(total_price) FROM bookings")
    avg_price = cursor.fetchone()[0]
    print(f"\nŚrednia cena rezerwacji: {round(avg_price, 2)} PLN")
    
    cursor.execute("SELECT AVG(rating) FROM reviews")
    avg_rating = cursor.fetchone()[0]
    if avg_rating is not None:
        print(f"Średnia ocena w opiniach: {round(avg_rating, 2)}/10")
    
    cursor.execute("""
    SELECT d.name, d.country, COUNT(*) as booking_count
    FROM bookings b
    JOIN tours t ON b.tour_id = t.tour_id
    JOIN destinations d ON t.destination_id = d.destination_id
    GROUP BY d.destination_id
    ORDER BY booking_count DESC
    LIMIT 5
    """)
    top_destinations = cursor.fetchall()
    
    print("\nTop 5 najpopularniejszych destynacji:")
    for i, (name, country, count) in enumerate(top_destinations, 1):
        print(f"{i}. {name}, {country} - {count} rezerwacji")
    
    conn.close()

show_database_stats()

Statystyki bazy danych biura podróży:
----------------------------------------
Clients: 297 rekordów
Destinations: 30 rekordów
Hotels: 145 rekordów
Tours: 89 rekordów
Employees: 50 rekordów
Bookings: 572 rekordów
Payments: 1126 rekordów
Reviews: 299 rekordów

Średnia cena rezerwacji: 14624.23 PLN
Średnia ocena w opiniach: 7.92/10

Top 5 najpopularniejszych destynacji:
1. Bariloche, Argentyna - 43 rezerwacji
2. Park Narodowy Kruger, Republika Południowej Afryki - 39 rezerwacji
3. Sedona, Stany Zjednoczone - 39 rezerwacji
4. Sinai Desert oazy (np. Dahab), Egipt - 34 rezerwacji
5. Wyspa Lord Howe, Australia - 33 rezerwacji


## Przykładowe zapytania SQL

Poniżej przedstawiono kilka przykładowych zapytań SQL, które można wykorzystać w zadaniu RAG.

In [32]:
def run_example_queries():
    """
    Wykonuje przykładowe zapytania SQL i wyświetla wyniki.
    """
    conn = sqlite3.connect(db_file)
    
    queries = [
        {"name": "Top 5 najlepiej ocenianych hoteli", "query": """
        SELECT h.name, h.stars, d.name as destination, d.country, AVG(r.rating) as avg_rating, COUNT(r.review_id) as reviews_count
        FROM hotels h
        JOIN destinations d ON h.destination_id = d.destination_id
        JOIN bookings b ON h.hotel_id = b.hotel_id
        JOIN reviews r ON b.booking_id = r.booking_id
        GROUP BY h.hotel_id
        HAVING COUNT(r.review_id) > 0
        ORDER BY avg_rating DESC
        LIMIT 5
        """},
        
        {"name": "Miesięczne przychody z rezerwacji", "query": """
        SELECT strftime('%Y-%m', booking_date) as month, SUM(total_price) as total_revenue
        FROM bookings
        WHERE status != 'Wstępna'
        GROUP BY month
        ORDER BY month
        """},
        
        {"name": "Klienci z największą liczbą rezerwacji", "query": """
        SELECT c.first_name, c.last_name, c.email, COUNT(b.booking_id) as bookings_count, SUM(b.total_price) as total_spent
        FROM clients c
        JOIN bookings b ON c.client_id = b.client_id
        GROUP BY c.client_id
        ORDER BY bookings_count DESC
        LIMIT 10
        """},
        
        {"name": "Najpopularniejsze wycieczki według liczby rezerwacji", "query": """
        SELECT t.name, d.name as destination, d.country, t.duration_days, 
               COUNT(b.booking_id) as booking_count, AVG(r.rating) as avg_rating
        FROM tours t
        JOIN destinations d ON t.destination_id = d.destination_id
        LEFT JOIN bookings b ON t.tour_id = b.tour_id
        LEFT JOIN reviews r ON b.booking_id = r.booking_id
        GROUP BY t.tour_id
        ORDER BY booking_count DESC
        LIMIT 10
        """},
        
        {"name": "Pracownicy z największą sprzedażą", "query": """
        SELECT e.first_name, e.last_name, e.position, COUNT(b.booking_id) as sales_count, SUM(b.total_price) as sales_total
        FROM employees e
        JOIN bookings b ON e.employee_id = b.employee_id
        GROUP BY e.employee_id
        ORDER BY sales_total DESC
        LIMIT 5
        """}
    ]
    
    for query_info in queries:
        print(f"\n{query_info['name']}:")
        print("-" * len(query_info['name']) * 2)
        
        try:
            df = pd.read_sql_query(query_info['query'], conn)
            display(df.head(10))
        except Exception as e:
            print(f"Błąd wykonania zapytania: {e}")
    
    conn.close()

run_example_queries()


Top 5 najlepiej ocenianych hoteli:
------------------------------------------------------------------


,name,stars,destination,country,avg_rating,reviews_count
0,Business Inn Iguazú,4,Wodospady Iguazú,Argentyna/Brazylia,9.0,1
1,Denali Business Suites,4,Park Narodowy Denali,Stany Zjednoczone,9.0,1
2,Alaska Denali Lodge,5,Park Narodowy Denali,Stany Zjednoczone,9.0,2
3,Eco Komodo Retreat,4,Luźny Rajski Archipelag Komodo,Indonezja,9.0,2
4,Komodo Breeze Resort,5,Luźny Rajski Archipelag Komodo,Indonezja,9.0,2



Miesięczne przychody z rezerwacji:
------------------------------------------------------------------


,month,total_revenue
0,2023-04,106287.01
1,2023-05,339347.41
2,2023-06,402107.91
3,2023-07,420502.64
4,2023-08,479305.88
5,2023-09,281934.81
6,2023-10,200063.38
7,2023-11,339219.35
8,2023-12,339500.28
9,2024-01,245452.45



Klienci z największą liczbą rezerwacji:
----------------------------------------------------------------------------


,first_name,last_name,email,bookings_count,total_spent
0,Piotr,Gabryszak,piotr.gabryszak@o2.pl,4,72483.33
1,Jan,Pronobis,jan.pronobis@wp.pl,4,60342.64
2,Anita,Moneta,anita.moneta@o2.pl,4,104314.43
3,Kacper,Miękina,kacper.miękina@yahoo.com,4,83756.79
4,Damian,Maraszek,damian.maraszek@gmail.com,4,49322.95
5,Miłosz,Smoczyk,miłosz.smoczyk@yahoo.com,4,92844.60
6,Oliwier,Szymańczak,oliwier.szymańczak@interia.pl,4,26909.06
7,Aleks,Janke,aleks.janke@onet.pl,4,23606.26
8,Igor,Woźniczko,igor.woźniczko@wp.pl,4,74021.65
9,Damian,Oleksak,damian.oleksak@interia.pl,4,51944.06



Najpopularniejsze wycieczki według liczby rezerwacji:
--------------------------------------------------------------------------------------------------------


,name,destination,country,duration_days,booking_count,avg_rating
0,Rodzinne Odkrywanie Bariloche,Bariloche,Argentyna,4,15,8.000000
1,Kulturowa eksploracja Luang Prabang,Luang Prabang,Laos,3,12,7.833333
2,Relaks i wellness na rajskiej plaży Zanzibaru,Zanzibar,Tanzania,7,12,7.800000
3,Kulturowa Podróż przez Trulli i Historię,Alberobello,Włochy,3,12,7.500000
4,Relaks i Wellness na Lord Howe,Wyspa Lord Howe,Australia,4,12,8.000000
5,Ekstremalna Przygoda Off-road i Hiking,Sedona,Stany Zjednoczone,4,11,8.500000
6,Relaks i Wellness w Sedonie,Sedona,Stany Zjednoczone,5,11,7.833333
7,Kulturowa Podróż po Wyspie Lord Howe,Wyspa Lord Howe,Australia,2,11,8.166667
8,Rodzinne Odkrywanie Cinque Terre,Cinque Terre,Włochy,3,10,7.800000
9,Przygoda na szlakach Bariloche,Bariloche,Argentyna,6,10,8.428571



Pracownicy z największą sprzedażą:
------------------------------------------------------------------


,first_name,last_name,position,sales_count,sales_total
0,Jan,Matyka,Doradca turystyczny,59,914555.19
1,Konstanty,Łachut,Specjalista ds. rezerwacji,46,850113.06
2,Janina,Salwin,Doradca turystyczny,49,738684.98
3,Adrian,Melaniuk,Doradca turystyczny,57,738430.44
4,Rafał,Kroczak,Doradca turystyczny,50,738017.11


## Podsumowanie

Wygenerowana baza danych zawiera syntetyczne dane operacyjne biura podróży, obejmujące informacje o klientach, destynacjach, hotelach, wycieczkach, rezerwacjach, płatnościach i opiniach. Baza została zapisana w pliku SQLite w folderze `008-03. RAG data`.

Korzystając z tej bazy danych, studenci mogą tworzyć system RAG SQL, który będzie odpowiadał na pytania dotyczące zawartości bazy. Przykłady pytań, na które system może odpowiadać:
- Jakie są najpopularniejsze destynacje?
- Którzy klienci wydali najwięcej pieniędzy?
- Jakie hotele mają najlepsze oceny?
- Jaki jest średni czas trwania wycieczek do danego kraju?
- Jakie są trendy w liczbie rezerwacji w poszczególnych miesiącach?

Wszystkie dane zostały wygenerowane z wykorzystaniem LLM (GPT-4.1-mini) dla treści opisowych oraz losowo generowanych wartości liczbowych z zachowaniem odpowiednich proporcji i zakresów.